## Selective Search Example
- Ref : https://www.pyimagesearch.com/2020/06/29/opencv-selective-search-for-object-detection/

In [1]:
import cv2
import numpy as np
import time

### Parameter

In [11]:
FPATH = '1.png'
OPTION = 'quality' # or 'quality'

### Prepare Input Data

In [3]:
# read image
im = cv2.imread(FPATH)

# resize image (for faster processing)
im = cv2.resize(im, (300, 300))

### Selective Search based on OpenCV

In [12]:
# create selective search segmentation object using default parameters
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

# set input image
ss.setBaseImage(im)

# switch option (should be after setBaseImage)
if OPTION == 'fast':
    ss.switchToSelectiveSearchFast()
else:
    ss.switchToSelectiveSearchQuality()

In [13]:
# run selective search
t1 = time.time()
rects = ss.process()
t2 = time.time()

In [14]:
print('Runtime for %s image: %.2f sec' % (im.shape, t2 - t1))
print('Region found: %d' % len(rects))

Runtime for (300, 300, 3) image: 2.72 sec
Region found: 5650


### Visualization result

In [15]:
cv2.namedWindow('Result', cv2.WINDOW_NORMAL)
image_vis = im.copy()

# Overlay found proposals
for ind in np.random.choice(len(rects), 10, replace=False):
    x, y, w, h = rects[ind]
    color = [np.random.randint(0, 255) for j in range(3)]
    cv2.rectangle(image_vis, (x, y), (x+w, y+h), color, 3)

cv2.imshow('Result', image_vis)
cv2.waitKey(0)
cv2.destroyAllWindows()